In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tqdm import tqdm
import tarfile
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

print('-------------------')
print('|     lab1         |')
print('-------------------')

# 데이터 다운로드 및 추출
url = "https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.tgz"
urllib.request.urlretrieve(url, "housing.tgz")

tar = tarfile.open("housing.tgz")
tar.extractall()
tar.close()

# 데이터 로드
housing = pd.read_csv("housing.csv")

print(housing.describe())
print(housing.info())


-------------------
|     lab1         |
-------------------
          longitude      latitude  housing_median_age   total_rooms  \
count  20640.000000  20640.000000        20640.000000  20640.000000   
mean    -119.569704     35.631861           28.639486   2635.763081   
std        2.003532      2.135952           12.585558   2181.615252   
min     -124.350000     32.540000            1.000000      2.000000   
25%     -121.800000     33.930000           18.000000   1447.750000   
50%     -118.490000     34.260000           29.000000   2127.000000   
75%     -118.010000     37.710000           37.000000   3148.000000   
max     -114.310000     41.950000           52.000000  39320.000000   

       total_bedrooms    population    households  median_income  \
count    20433.000000  20640.000000  20640.000000   20640.000000   
mean       537.870553   1425.476744    499.539680       3.870671   
std        421.385070   1132.462122    382.329753       1.899822   
min          1.000000      

In [3]:
housing


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


### 이상치 탐지

1. iqr 값을 계산해서 각 이상치 값이 있는 데이터의 인덱스를 찾아낸다.
2. 인덱스의 중복을 set으로 제거한 후, 데이터에서 인덱스를 제거한다.


In [4]:
def AnomalyDelete(df: pd.DataFrame):
    col = df.columns[:-1]
    stack = []

    quantile_25 = df[col].quantile(0.25)
    quantile_75 = df[col].quantile(0.75)
    iqr = quantile_75 - quantile_25
    iqr_weight = iqr * 1.5

    for i in col:
        lowest_val = quantile_25[i] - iqr_weight[i]
        highest_val = quantile_75[i] + iqr_weight[i]
        outlier_idx = df[(df[i] < lowest_val) | (df[i] > highest_val)].index
        stack.extend(outlier_idx)

    return list(set(stack))


def delHandler(X_train: pd.DataFrame, y_train: pd.Series, idxs):
    X_train = X_train.drop(index=idxs, axis=0)
    y_train = y_train.drop(index=idxs, axis=0)
    return X_train, y_train


def AnomalyHandler(X_train: pd.DataFrame, y_train):
    idxs = AnomalyDelete(X_train)
    X_train, y_train = delHandler(X_train, y_train, idxs)
    return X_train, y_train

### 전처리
1. label 인코딩을 진행한다.
2. 스탠다드 스케일링을 진행한다.
3. train_df의 경우에는 fit_transform, test데이터는 transform을 진행한다. 

In [5]:
class CombinedAttributesAdder():

    def __init__(self):
        self.le = LabelEncoder()
        self.sk = StandardScaler()

    def fit_and_transform(self, train_df, test_df):
        train_df, test_df = self._DataHandler(train_df, test_df)
        return train_df, test_df

    def _Encoder(self, train_df, test_df):
        feature = 'ocean_proximity'
        train_df[feature] = self.le.fit_transform(train_df[feature])
        test_df[feature] = self.le.transform(test_df[feature])
        return train_df, test_df

    def _Norm(self, train_df, test_df):
        numeric_features = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
                            'total_bedrooms', 'population', 'households']
        for feature in numeric_features:
            train_df[feature] = self.sk.fit_transform(train_df[[feature]])
            test_df[feature] = self.sk.transform(test_df[[feature]])
        return train_df, test_df

    def _DataHandler(self, train_df: pd.DataFrame, test_df: pd.DataFrame):
        train_df, test_df = self._Encoder(train_df, test_df)
        train_df, test_df = self._Norm(train_df, test_df)
        return train_df, test_df

In [6]:
housing = housing.sample(frac=1).reset_index(drop=True)

train_df = housing.iloc[:18000].copy()
test_df = housing.iloc[18000:].copy()

handler = CombinedAttributesAdder()
train_df, test_df = handler.fit_and_transform(train_df=train_df, test_df=test_df)

print(train_df.head())
print(test_df.head())


   longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0   1.234933 -1.375096            0.428748    -0.534476       -0.642579   
1   0.680907 -0.705833            1.858687    -0.483703       -0.265257   
2  -1.694917  2.406475            0.031543    -0.457631       -0.571386   
3   0.526179 -0.668392            0.508189    -0.468151       -0.545282   
4  -1.625040  1.367479           -1.636720     1.624975        1.305730   

   population  households  median_income  median_house_value  ocean_proximity  
0   -0.347003   -0.607841         3.4583            112700.0                4  
1    0.031940   -0.244467         3.3326            167600.0                0  
2   -0.591425   -0.599998         3.2891             93100.0                1  
3   -0.506141   -0.443146         4.1806            209000.0                0  
4    1.355162    1.308365         3.8616            195100.0                0  
       longitude  latitude  housing_median_age  total_rooms  total_be

In [7]:
train_x = train_df.drop(['median_house_value'], axis=1)
test_x = test_df.drop(['median_house_value'], axis=1)

train_y = train_df['median_house_value']
test_y = test_df['median_house_value']

train_x, train_y = AnomalyHandler(train_x, train_y)

train_x["income_cat"] = pd.cut(train_df["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])
test_x["income_cat"] = pd.cut(test_df["median_income"],
                              bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                              labels=[1, 2, 3, 4, 5])

train_x = pd.get_dummies(train_x, columns=['income_cat'])
test_x = pd.get_dummies(test_x, columns=['income_cat'])

### GridSearchCV를 이용한 하이퍼파라미터 최적화

In [8]:
xgb_reg = xgb.XGBRegressor(random_state=123)
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [5, 7, 10],
}
grid_search = GridSearchCV(xgb_reg, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(train_x, train_y)
print("Best parameters: ", grid_search.best_params_)
best_model = grid_search.best_estimator_

test_predictions = best_model.predict(test_x)
test_mse = mean_squared_error(test_y, test_predictions)
test_rmse = np.sqrt(test_mse)
test_r2 = r2_score(test_y, test_predictions)

print("테스트 세트 성능:")
print("RMSE:", test_rmse)
print("R2:", test_r2)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] END ...learning_rate=0.01, max_depth=5, n_estimators=50; total time=   0.1s
[CV] END ...learning_rate=0.01, max_depth=5, n_estimators=50; total time=   0.1s
[CV] END ...learning_rate=0.01, max_depth=5, n_estimators=50; total time=   0.1s
[CV] END ...learning_rate=0.01, max_depth=5, n_estimators=50; total time=   0.0s
[CV] END ...learning_rate=0.01, max_depth=5, n_estimators=50; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=5, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=5, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=5, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=5, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=5, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=5, n_estimators=200; total time=   0.3s
[CV] END ..learning_rate=0.01, max_depth=5, n_e

### 베이지안 최적화로 object function optimize

In [9]:
from hyperopt import hp
from sklearn.model_selection import cross_val_score
from hyperopt import STATUS_OK, fmin, tpe, Trials
import numpy as np
import xgboost as xgb

xgb_search_space = {
    'max_depth': hp.quniform('max_depth', 5, 20, 1),
    'min_child_weight': hp.quniform('min_child_weight', 1, 2, 1),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
}

In [10]:
def objective_func(search_space):
    xgb_reg = xgb.XGBRegressor(
        n_estimators=100,
        max_depth=int(search_space['max_depth']),
        min_child_weight=int(search_space['min_child_weight']),
        learning_rate=search_space['learning_rate'],
        colsample_bytree=search_space['colsample_bytree'],
        random_state=123
    )
    mse = cross_val_score(xgb_reg, train_x, train_y, scoring='neg_mean_squared_error', cv=3)
    mse = -1.0 * np.mean(mse)

    return {'loss': mse, 'status': STATUS_OK}


trial_val = Trials()
best = fmin(
    fn=objective_func,
    space=xgb_search_space,
    algo=tpe.suggest,
    max_evals=20,
    trials=trial_val,
    rstate=np.random.default_rng(seed=9)
)

print('best:', best)


100%|██████████| 20/20 [00:32<00:00,  1.62s/trial, best loss: 2270824682.5156717]
best: {'colsample_bytree': np.float64(0.8637740285716389), 'learning_rate': np.float64(0.10657919742273766), 'max_depth': np.float64(8.0), 'min_child_weight': np.float64(2.0)}


### best param

colsample_bytree = 0.77084
learning_rate = 0.0925
max_depth = 9
min_child_weight = 2


In [11]:
xgb_reg = xgb.XGBRegressor(
    n_estimators=200,
    max_depth=8,
    min_child_weight=2,
    learning_rate=0.09025,
    random_state=123
)

In [12]:
model = xgb_reg.fit(train_x, train_y)

In [13]:
test_predictions = model.predict(test_x)
test_mse = mean_squared_error(test_y, test_predictions)
test_rmse = np.sqrt(test_mse)
test_r2 = r2_score(test_y, test_predictions)

print("테스트 세트 성능:")
print("RMSE:", test_rmse)
print("R2:", test_r2)

테스트 세트 성능:
RMSE: 49346.21488033891
R2: 0.8172342280018438


ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

In [15]:
test_y


18000    123200.0
18001    173200.0
18002    255800.0
18003    125700.0
18004    165800.0
           ...   
20635    286300.0
20636    251900.0
20637     99800.0
20638    147400.0
20639    137500.0
Name: median_house_value, Length: 2640, dtype: float64